In [65]:
import pandas as pd # data modeling/transformation
import nflreadpy as nfl # nfl data api wrapper
from sklearn.model_selection import train_test_split # machine learning module
from sklearn.ensemble import RandomForestClassifier # machine learning module
from sklearn.metrics import accuracy_score # machine learning module
import matplotlib.pyplot as plt # for plotting
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import seaborn as sns

# NOTE - CONVERT ALL DF'S TO PANDAS AS NFLREADPY USES POLARS BY DEFAULT

In [66]:
# define timeframe and teams
years = [2023, 2024, 2025]

# Load pbp and filter data for seasons listed
pbp = nfl.load_pbp(seasons=years).to_pandas()

# Load and filter historical game data
games = nfl.load_schedules(years).to_pandas()

In [ ]:

# Aggregate team performance per game
team_game_stats = (
    pbp.groupby(['game_id', 'posteam'])
    .agg({
        'epa': 'mean',            # expected points added per play
        'success': 'mean',        # success rate
        'yards_gained': 'sum',
        'touchdown': 'sum',
        'interception': 'sum',
        'fumble_lost': 'sum'
    })
    .reset_index()
    .rename(columns={'posteam': 'team'})
)

def get_opponent(row):
    if row['posteam'] == row['home_team']:
        return row['away_team']
    else:
        return row['home_team']


            game_id  season game_type  week     gameday   weekday gametime  \
0    2023_01_DET_KC    2023       REG     1  2023-09-07  Thursday    20:20   
1   2023_01_CAR_ATL    2023       REG     1  2023-09-10    Sunday    13:00   
2   2023_01_HOU_BAL    2023       REG     1  2023-09-10    Sunday    13:00   
3   2023_01_CIN_CLE    2023       REG     1  2023-09-10    Sunday    13:00   
4   2023_01_JAX_IND    2023       REG     1  2023-09-10    Sunday    13:00   
5    2023_01_TB_MIN    2023       REG     1  2023-09-10    Sunday    13:00   
6    2023_01_TEN_NO    2023       REG     1  2023-09-10    Sunday    13:00   
7    2023_01_SF_PIT    2023       REG     1  2023-09-10    Sunday    13:00   
8   2023_01_ARI_WAS    2023       REG     1  2023-09-10    Sunday    13:00   
9    2023_01_GB_CHI    2023       REG     1  2023-09-10    Sunday    16:25   
10   2023_01_LV_DEN    2023       REG     1  2023-09-10    Sunday    16:25   
11  2023_01_MIA_LAC    2023       REG     1  2023-09-10    Sunda

In [68]:
# Merge pbp and games data into single dataframe for machine learning

# Merge with game info
team_stats = team_game_stats.merge(games[['game_id', 'season', 'week', 'home_team', 'away_team', 'home_score', 'away_score']], on='game_id')

def get_opponent(row):
    if row['team'] == row['home_team']:
        return row['away_team']
    else:
        return row['home_team']

team_stats['opponent'] = team_stats.apply(get_opponent, axis=1)

team_stats[['season', 'week', 'team', 'opponent']].head(10)

# Feature Engineering
team_stats['win'] = team_stats.apply(
    lambda row: 1 if (
        (row['team'] == row['home_team'] and row['home_score'] > row['away_score']) or
        (row['team'] == row['away_team'] and row['away_score'] > row['home_score'])
    ) else 0,
    axis=1
)


In [69]:
# Create rolling average feature to capture momentum - in this case a 3 week rolling average

team_stats = team_stats.sort_values(['team', 'season', 'week'])
team_stats['avg_points_last3'] = team_stats.groupby('team')['yards_gained'].transform(lambda x: x.rolling(3, min_periods=1).mean())


In [70]:
X = team_stats[['yards_gained', 'epa', 'touchdown', 'interception']]
y = team_stats['win']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

preds = model.predict(X_test)
print('Accuracy:', accuracy_score(y_test, preds))


Accuracy: 0.6838235294117647


In [71]:
corr = team_stats.corr(numeric_only=True)['win'].sort_values(ascending=False)

In [72]:
from sklearn.inspection import permutation_importance

result = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=42)
importance_df = pd.DataFrame({
    'feature': X.columns,
    'importance': result.importances_mean
}).sort_values(by='importance', ascending=False)

print(importance_df)

        feature  importance
1           epa    0.080147
2     touchdown    0.053309
3  interception    0.026103
0  yards_gained    0.008088


In [73]:
'''
# --- 1️⃣ Correlation with Win ---
corr = team_stats.corr(numeric_only=True)['win'].sort_values(ascending=False)

plt.figure(figsize=(10,4))
corr.drop('win').plot(kind='bar', color='skyblue')
plt.title('Correlation of Features with Win')
plt.ylabel('Correlation coefficient')
plt.xticks(rotation=45, ha='right')
plt.show()

# --- 2️⃣ Heatmap of All Feature Correlations ---
plt.figure(figsize=(10,8))
sns.heatmap(team_stats.corr(numeric_only=True), cmap='coolwarm', center=0, annot=False)
plt.title("Feature Correlation Heatmap")
plt.show()

# --- 3️⃣ Model Feature Importance (if model already trained) ---
importances = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)

plt.figure(figsize=(10,4))
importances.plot(kind='bar', color='orange')
plt.title('Feature Importance (Random Forest)')
plt.ylabel('Importance')
plt.xticks(rotation=45, ha='right')
plt.show()

# --- 4️⃣ Permutation Importance (more robust) ---
result = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=42)
perm_df = pd.DataFrame({
    'feature': X.columns,
    'importance': result.importances_mean
}).sort_values(by='importance', ascending=False)

plt.figure(figsize=(10,4))
sns.barplot(data=perm_df, x='importance', y='feature', palette='viridis')
plt.title('Permutation Importance (Impact on Model Accuracy)')
plt.xlabel('Mean importance across 10 shuffles')
plt.ylabel('')
plt.show()

# --- 5️⃣ Combine summary (optional) ---
summary = pd.DataFrame({
    'Correlation_with_Win': corr.drop('win'),
    'Model_Importance': importances,
    'Permutation_Importance': perm_df.set_index('feature')['importance']
}).sort_values(by='Model_Importance', ascending=False)

display(summary.round(3))
'''

'\n# --- 1️⃣ Correlation with Win ---\ncorr = team_stats.corr(numeric_only=True)[\'win\'].sort_values(ascending=False)\n\nplt.figure(figsize=(10,4))\ncorr.drop(\'win\').plot(kind=\'bar\', color=\'skyblue\')\nplt.title(\'Correlation of Features with Win\')\nplt.ylabel(\'Correlation coefficient\')\nplt.xticks(rotation=45, ha=\'right\')\nplt.show()\n\n# --- 2️⃣ Heatmap of All Feature Correlations ---\nplt.figure(figsize=(10,8))\nsns.heatmap(team_stats.corr(numeric_only=True), cmap=\'coolwarm\', center=0, annot=False)\nplt.title("Feature Correlation Heatmap")\nplt.show()\n\n# --- 3️⃣ Model Feature Importance (if model already trained) ---\nimportances = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)\n\nplt.figure(figsize=(10,4))\nimportances.plot(kind=\'bar\', color=\'orange\')\nplt.title(\'Feature Importance (Random Forest)\')\nplt.ylabel(\'Importance\')\nplt.xticks(rotation=45, ha=\'right\')\nplt.show()\n\n# --- 4️⃣ Permutation Importance (more robust

In [74]:
team_stats = team_stats.sort_values(['team', 'season', 'week'])
team_stats['epa_last3'] = team_stats.groupby('team')['epa'].transform(lambda x: x.rolling(3, min_periods=1).mean())
team_stats['epa_last5'] = team_stats.groupby('team')['epa'].transform(lambda x: x.rolling(5, min_periods=1).mean())

In [75]:
defensive_epa = team_stats.groupby('opponent')['epa'].mean().rename('def_epa_allowed')
team_stats = team_stats.merge(defensive_epa, left_on='team', right_index=True, how='left')
team_stats['epa_diff'] = team_stats['epa'] - team_stats['def_epa_allowed']

print(team_stats.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1356 entries, 0 to 1355
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   game_id           1356 non-null   object 
 1   team              1356 non-null   object 
 2   epa               1356 non-null   float64
 3   success           1356 non-null   float64
 4   yards_gained      1356 non-null   float64
 5   touchdown         1356 non-null   float64
 6   interception      1356 non-null   float64
 7   fumble_lost       1356 non-null   float64
 8   season            1356 non-null   int64  
 9   week              1356 non-null   int64  
 10  home_team         1356 non-null   object 
 11  away_team         1356 non-null   object 
 12  home_score        1356 non-null   float64
 13  away_score        1356 non-null   float64
 14  opponent          1356 non-null   object 
 15  win               1356 non-null   int64  
 16  avg_points_last3  1356 non-null   float64
 17  

In [76]:

# Define features and target
X = team_stats[['epa_diff']]
y = team_stats['win']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train logistic regression
model = LogisticRegression()
model.fit(X_train, y_train)

# Evaluate
preds = model.predict(X_test)
probs = model.predict_proba(X_test)[:,1]

print("Accuracy:", accuracy_score(y_test, preds))
print("ROC AUC:", roc_auc_score(y_test, probs))






Accuracy: 0.7058823529411765
ROC AUC: 0.7997833152762729


In [77]:
epa_range = np.linspace(team_stats['epa_diff'].min(), team_stats['epa_diff'].max(), 100).reshape(-1,1)
#touchdowns = np.linspace(team_stats['touchdown'].min(), team_stats['touchdown'].max(), 100).reshape(-1,1)

#print(epa_range)
#print(touchdowns)

win_prob = model.predict_proba(epa_range)[:,1]

'''
plt.figure(figsize=(8,5))
plt.plot(epa_range, win_prob, color='crimson')
plt.title("Win Probability vs. EPA Differential")
plt.xlabel("EPA Differential (Team EPA - Opponent Avg EPA Allowed)")
plt.ylabel("Predicted Win Probability")
plt.grid(True)
plt.show()
'''


/Users/jordanbormann/football/.venvpy310/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


'\nplt.figure(figsize=(8,5))\nplt.plot(epa_range, win_prob, color=\'crimson\')\nplt.title("Win Probability vs. EPA Differential")\nplt.xlabel("EPA Differential (Team EPA - Opponent Avg EPA Allowed)")\nplt.ylabel("Predicted Win Probability")\nplt.grid(True)\nplt.show()\n'

In [78]:
coef = model.coef_[0][0]
intercept = model.intercept_[0]
print(f"Logistic Coefficient for epa_diff: {coef:.3f}")
print(f"Intercept: {intercept:.3f}")


Logistic Coefficient for epa_diff: 6.837
Intercept: 0.012


In [ ]:
# Range of EPA differences (-0.5 to +0.5 covers realistic game-to-game spread)
epa_diff = np.linspace(-0.5, 0.5, 200)

# Logistic function for win probability
prob_win = 1 / (1 + np.exp(-(intercept + coef * epa_diff)))

'''
# Plot
plt.figure(figsize=(8,5))
plt.plot(epa_diff, prob_win, linewidth=2)
plt.axhline(0.5, color='gray', linestyle='--', alpha=0.6)
plt.axvline(0, color='gray', linestyle='--', alpha=0.6)

plt.title("Win Probability vs EPA Difference", fontsize=14)
plt.xlabel("EPA Difference (Team - Opponent)", fontsize=12)
plt.ylabel("Predicted Win Probability", fontsize=12)
plt.grid(alpha=0.3)
plt.show()
'''


'\n# Plot\nplt.figure(figsize=(8,5))\nplt.plot(epa_diff, prob_win, linewidth=2)\nplt.axhline(0.5, color=\'gray\', linestyle=\'--\', alpha=0.6)\nplt.axvline(0, color=\'gray\', linestyle=\'--\', alpha=0.6)\n\nplt.title("Win Probability vs EPA Difference", fontsize=14)\nplt.xlabel("EPA Difference (Team - Opponent)", fontsize=12)\nplt.ylabel("Predicted Win Probability", fontsize=12)\nplt.grid(alpha=0.3)\nplt.show()\n'

In [80]:
import datetime

games['gameday_date'] = pd.to_datetime(games['gameday']).dt.date
today = datetime.date.today()

this_week = games[
    (games['gameday_date'] >= today) &
    (games['gameday_date'] < today + datetime.timedelta(days=5))
]


In [81]:
# recent games only
recent_pbp = pbp[pbp['season'] >= 2023]

team_epa = (
    recent_pbp.groupby('posteam')['epa']
    .mean()
    .reset_index()
    .rename(columns={'epa': 'team_epa'})
)


In [82]:
def_epa = (
    recent_pbp.groupby('defteam')['epa']
    .mean()
    .reset_index()
    .rename(columns={'defteam': 'team', 'epa': 'def_epa'})
)

In [ ]:
team_stats = pd.merge(team_epa, def_epa, left_on='posteam', right_on='team', how='left')

team_stats = team_stats[['posteam', 'team_epa', 'def_epa']]
team_stats.rename(columns={'posteam': 'team'}, inplace=True)
#team_stats.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   posteam   32 non-null     object 
 1   team_epa  32 non-null     float64
 2   team      32 non-null     object 
 3   def_epa   32 non-null     float64
dtypes: float64(2), object(2)
memory usage: 1.1+ KB
None


In [84]:

pred_df = this_week.copy()

pred_df = pred_df.merge(team_stats, left_on='home_team', right_on='team', how='left')

pred_df.rename(columns={'team_epa': 'home_epa', 'def_epa': 'home_def_epa'}, inplace=True)
pred_df = pred_df.merge(team_stats, left_on='away_team', right_on='team', how='left')
pred_df.rename(columns={'team_epa': 'away_epa', 'def_epa': 'away_def_epa'}, inplace=True)


# Compute offensive EPA difference (home - away)
pred_df['epa_diff'] = (pred_df['home_epa'] - pred_df['away_epa'])


In [ ]:
pred_df['win_prob_home'] = 1 / (1 + np.exp(-(intercept + coef * pred_df['epa_diff'])))
pred_df['model_winner'] = np.where(pred_df['win_prob_home'] > 0.5, pred_df['home_team'], pred_df['away_team'])

pred_df['win_prob_away'] = 1 - pred_df['win_prob_home']

'''
pred_df['win_prob_home'] = pred_df['win_prob_home'].apply(lambda x: f'{x:.2%}')
pred_df['win_prob_away'] = pred_df['win_prob_away'].apply(lambda x: f'{x:.2%}')
'''

final = pred_df[['gameday', 'home_team', 'away_team', 'win_prob_home', 'win_prob_away','home_moneyline','away_moneyline','model_winner']]

final.to_csv("weekly_prediction.csv")



In [ ]:
# EV RANKING - NEEDS WORK

'''
numeric_cols = ['home_moneyline', 'away_moneyline', 'win_prob_home', 'win_prob_away']
for col in numeric_cols:
    pred_df[col] = pd.to_numeric(pred_df[col], errors='coerce')



def rank_bets(pred_df):
    # Convert moneyline to implied probability
    def moneyline_to_prob(ml):
        if ml > 0:
            return 100 / (ml + 100)
        else:
            return -ml / (-ml + 100)

    pred_df['home_implied_prob'] = pred_df['home_moneyline'].apply(moneyline_to_prob)
    pred_df['away_implied_prob'] = pred_df['away_moneyline'].apply(moneyline_to_prob)

    # Compute edge
    pred_df['home_edge'] = pred_df['win_prob_home'] - pred_df['home_implied_prob']
    pred_df['away_edge'] = pred_df['win_prob_away'] - pred_df['away_implied_prob']

    # Compute expected value in dollars
    def expected_value(prob, ml):
        if ml > 0:
            payout = ml / 100
        else:
            payout = 100 / -ml
        return prob * payout - (1 - prob)

    pred_df['home_ev'] = pred_df.apply(lambda row: expected_value(row['win_prob_home'], row['home_moneyline']), axis=1)
    pred_df['away_ev'] = pred_df.apply(lambda row: expected_value(row['win_prob_away'], row['away_moneyline']), axis=1)

    # Melt into long format for easier ranking
    home = pred_df[['home_team', 'win_prob_home', 'home_implied_prob', 'home_edge', 'home_ev']].rename(
        columns={'home_team': 'team', 'win_prob_home': 'model_prob', 'home_implied_prob': 'implied_prob',
                 'home_edge': 'edge', 'home_ev': 'ev'}
    )

    away = pred_df[['away_team', 'win_prob_away', 'away_implied_prob', 'away_edge', 'away_ev']].rename(
        columns={'away_team': 'team', 'win_prob_away': 'model_prob', 'away_implied_prob': 'implied_prob',
                 'away_edge': 'edge', 'away_ev': 'ev'}
    )

    ev_df = pd.concat([home, away]).reset_index(drop=True)

    # Rank by expected value (descending)
    ev_df = ev_df.sort_values('ev', ascending=False).reset_index(drop=True)
    ev_df['rank'] = ev_df['ev'].rank(method='first', ascending=False).astype(int)

    # Filter only positive EV for "safe" bets (optional)
    ev_df = ev_df[ev_df['ev'] > 0]

    return ev_df

ranked_bets = rank_bets(pred_df)
ranked_bets[['team','model_prob','implied_prob','edge','ev','rank']]

'''


,team,model_prob,implied_prob,edge,ev,rank
0,TEN,0.370052,0.105263,0.264789,2.515493,1
1,WAS,0.473226,0.153846,0.319379,2.075967,2
2,MIA,0.586201,0.227273,0.358928,1.579284,3
3,CLE,0.431139,0.250000,0.181139,0.724555,4
4,DAL,0.582938,0.393701,0.189237,0.480661,5
5,SF,0.640736,0.512195,0.128541,0.250961,6
6,NYG,0.256926,0.238095,0.018831,0.079089,7
7,NYJ,0.307905,0.294118,0.013787,0.046876,8
8,GB,0.628855,0.618321,0.010535,0.017038,9
